In [33]:
import pandas as pd
import numpy as np


In [42]:
Chicago_zipcodes = [60660, 60626, 60645, 60659, 60646, 60631, 60656, 60630, 60625, 60634, 60641, 
                    60618, 60657, 60613, 60640, 60651, 60622, 60614, 60647, 60639, 60642, 60644, 
                    60624, 60612, 60601, 60602, 60603, 60604, 60605, 60606, 60607, 60610, 60611, 
                    60661, 60654, 60628, 60608, 60616, 60653, 60615, 60609, 60632, 60638, 60629, 
                    60636, 60621, 60637, 60652, 60620, 60619, 60628, 60643, 60655, 60827, 60623, 
                    60649, 60617, 60633]


In [242]:
#merge yr info, create columns create new df out of raw_data

# read in data and sort by zipcode
raw_data = pd.read_csv('output_dataset.csv')
raw_data.sort_values(by=['zip code tabulation area'], inplace=True)

# create a new df with just the y features, filter new df for just Chicago zipcodes

test_df = pd.DataFrame().assign(Zipcode=raw_data['zip code tabulation area'], Year=raw_data['year'], Income=raw_data['med_income'], 
                                High_Ed=raw_data['perc_hs_grad'], Home_Val=raw_data['med_home_val'])

zipc = test_df[test_df['Zipcode'].isin(Chicago_zipcodes)]

#create df for 3 years for comparison (2012, 2016, 2019)

df_12 =zipc[zipc['Year'].isin([2012])]
df_12.columns = ['Zipcode', 'Year', 'Income12', 'High_Ed12', 'Home_Val12']


df_16 =zipc[zipc['Year'].isin([2016])]
df_16.columns = ['Zipcode', 'Year', 'Income16', 'High_Ed16', 'Home_Val16']

df_19 =zipc[zipc['Year'].isin([2019])]
df_19.columns = ['Zipcode', 'Year', 'Income19', 'High_Ed19', 'Home_Val19']

#Merge yr df to one df
test= pd.merge(df_12, df_16, on='Zipcode', how='inner')
final_df = pd.merge(test, df_19, on='Zipcode', how='inner')

final_df['Percent Change Income'] = 'NAN'
final_df['Percent Change Education'] = 'NAN'
final_df['Percent Change Home Value'] = 'NAN'
#final_df


In [247]:
# code for percentage change and addign columns
        
def income_change(df):
    '''
    '''
    a = (df['Income16'] - df['Income12'])/ df['Income12'] 
    b = (df['Income19'] - df['Income16'])/ df['Income16'] 
    c = a + b
    return c

def edu_percent(df): 
    a = (df['High_Ed16'] - df['High_Ed12'])/ df['High_Ed12'] 
    b = (df['High_Ed19'] - df['High_Ed16'])/ df['High_Ed16'] 
    c = (a + b) 
    return c

def home_val_change(df):
    '''
    '''
    a = (df['Home_Val16'] - df['Home_Val12'])/ df['Home_Val12'] 
    b = (df['Home_Val19'] - df['Home_Val16'])/ df['Home_Val16'] 
    c = a + b
    return c
    

In [248]:
# create 4 new calculated columns in df including calculated y value; 1 for gentrified and 0 for other


# average income percent change was about 22.3%, print(sum(income_percent)/57)

income_percent = income_change(final_df)

# average high school education change was about 26.7%, print(sum(ed_percent)/57)
ed_percent = edu_percent(final_df)

#average home percent change was 4.27%, print(sum(home_percent)/57)
home_percent = home_val_change(final_df)
    
final_df['Percent Change Income'] = income_percent.tolist()
final_df['Percent Change Education'] = ed_percent .tolist()
final_df['Percent Change Home Value'] = home_percent.tolist()


In [262]:
def y_val_func(df):
    '''
    '''
    y_lst = []
    income_count_lst = []
    ed_count_lst = []
    home_count_lst = []
   
    a = df['Percent Change Income'].tolist()
    b = df['Percent Change Education'].tolist()
    c = df['Percent Change Home Value'].tolist()

    for val in a:
        if val >= 0.25:
            income_count_lst.append(1)
        else:
            income_count_lst.append(0)
    for val in c:
        if val >= 0.10:
            home_count_lst.append(1)
        else:
            home_count_lst.append(0)
            
    for val in b:
        if val >= 0.40:
            ed_count_lst.append(1)
        else:
            ed_count_lst.append(0)
    
    in_arr = np.array(income_count_lst)
    home_arr = np.array(home_count_lst)
    ed_arr = np.array(ed_count_lst)
    fin_arr = np.add(in_arr, home_arr, ed_arr)
    
    for val in fin_arr: 
        if val >= 2:
            y_lst.append(1)
        else: 
            y_lst.append(0)
    return y_lst

In [263]:
y_val_lst = y_val_func(final_df)
final_df['Y_Value'] = y_val_lst 
final_df


,Zipcode,Year_x,Income12,High_Ed12,Home_Val12,Year_y,Income16,High_Ed16,Home_Val16,Year,Income19,High_Ed19,Home_Val19,Percent Change Income,Percent Change Education,Percent Change Home Value,Y_Value
0,60601.0,2012,80613.0,0.027913,465700.0,2016,103735.0,0.064878,463300.0,2019,103112.0,0.024112,519400.0,0.280821,0.695956,0.115934,1
1,60602.0,2012,81750.0,0.094880,330400.0,2016,85313.0,0.089724,375400.0,2019,191528.0,0.092460,392300.0,1.288588,-0.023851,0.181217,1
2,60603.0,2012,107056.0,0.062147,618100.0,2016,140500.0,0.008741,613400.0,2019,146250.0,0.158314,672700.0,0.353323,16.251807,0.089070,0
3,60604.0,2012,105984.0,0.056555,482600.0,2016,150125.0,0.035052,460400.0,2019,91750.0,0.080161,485900.0,0.027645,0.906721,0.009386,0
4,60605.0,2012,71362.0,0.063867,354400.0,2016,99725.0,0.041452,378100.0,2019,111438.0,0.049072,398900.0,0.514905,-0.167150,0.121885,1
5,60606.0,2012,101912.0,0.018079,327500.0,2016,119973.0,0.016153,393500.0,2019,140558.0,0.005684,402500.0,0.348802,-0.754632,0.224398,1
6,60607.0,2012,80501.0,0.061329,347000.0,2016,86476.0,0.055780,348700.0,2019,102726.0,0.063850,396400.0,0.262136,0.054197,0.141693,1
7,60608.0,2012,35603.0,0.237413,247800.0,2016,37574.0,0.207392,226700.0,2019,50133.0,0.200404,265400.0,0.389608,-0.160144,0.085561,0
8,60609.0,2012,34966.0,0.263696,192000.0,2016,32721.0,0.350154,157500.0,2019,37353.0,0.366426,181400.0,0.077355,0.374341,-0.027941,0
9,60610.0,2012,71986.0,0.081456,421700.0,2016,77094.0,0.067521,431300.0,2019,92130.0,0.062790,451900.0,0.265993,-0.241144,0.070528,0
